In [ ]:
!pip install transformers

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

from sklearn.model_selection import train_test_split
import pandas as pd
import torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 103.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.4 MB/s eta 0:00:00


In [ ]:
import json
import pandas as pd  

f = open('human_data.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)

human_df = pd.DataFrame(data)
f = open('ai.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
ai_df = pd.DataFrame(data)
ai_df.columns = ['text', 'label']
df_concat = pd.concat([human_df, ai_df])

# Shuffle the concatenated dataframe
df_shuffled = df_concat.sample(frac=1).reset_index(drop=True)

# Print the shuffled dataframe
#print(df_shuffled)
df = df_shuffled

In [ ]:
len(df_shuffled)

8159

In [ ]:
df_shuffled.columns

Index(['text', 'label'], dtype='object')

In [ ]:
data = df_shuffled

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.33, random_state=42)

In [ ]:
len(X_train)

5466

In [ ]:
X_val = X_train[0:1500]
X_train = X_train[1500:]
y_val = y_train[0:1500]
y_train = y_train[1500:]

In [ ]:
tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-turkish-128k-uncased')
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(X_val.tolist(), truncation=True, padding=True)

train_labels = y_train.tolist()
val_labels = y_val.tolist()


In [ ]:
train_dataset = torch.utils.data.TensorDataset(
        torch.tensor(train_encodings['input_ids']),
        torch.tensor(train_encodings['attention_mask']),
        torch.tensor(train_labels))
val_dataset = torch.utils.data.TensorDataset(
        torch.tensor(val_encodings['input_ids']),
        torch.tensor(val_encodings['attention_mask']),
        torch.tensor(val_labels))

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('dbmdz/bert-base-turkish-128k-cased', num_labels=2)
training_args= TrainingArguments(
    output_dir=f'./results_bert-base-turkish-128k-cased/fold/gpt2',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=f'./logs_bert-base-turkish-128k-cased/fold/gpt2',
    logging_steps=100)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=lambda data: {'input_ids': torch.stack([item[0] for item in data]),
                                'attention_mask': torch.stack([item[1] for item in data]),
                                'labels': torch.stack([item[2] for item in data])})
trainer.train()

Some weights of the model checkpoint at dbmdz/bert-base-turkish-128k-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification 

Step,Training Loss
100,0.628200
200,0.299500
300,0.129300
400,0.103400
500,0.134400
600,0.030400
700,0.008000


TrainOutput(global_step=744, training_loss=0.1795794668216859, metrics={'train_runtime': 1110.4873, 'train_samples_per_second': 10.714, 'train_steps_per_second': 0.67, 'total_flos': 3130495336673280.0, 'train_loss': 0.1795794668216859, 'epoch': 3.0})

In [ ]:
import torch
torch.save(model, 'gpt2_BERT.sav')

In [ ]:
# Make predictions on the test data
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True)
num_test_samples = len(test_encodings['input_ids'])
test_labels = [0] * num_test_samples

test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(test_labels))

predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)





In [ ]:
from sklearn.metrics import classification_report
print("Classification report:")
print(classification_report(y_test, preds))


Classification report:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      1302
           1       0.99      1.00      0.99      1391

    accuracy                           0.99      2693
   macro avg       0.99      0.99      0.99      2693
weighted avg       0.99      0.99      0.99      2693

